In [6]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

# U.S. STOCK MARKET RETURN

## ANNUAL

In [7]:
mkt_annual = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt_annual = mkt_annual['Mkt-RF'] + mkt_annual.RF
mkt_annual.index.name = "year"
mkt_annual.name = "market annual"
mkt_annual.index = mkt_annual.index.astype(str)
mkt_annual.tail()

year
2018   -0.0512
2019    0.3043
2020    0.2411
2021    0.2360
2022   -0.2017
Name: market annual, dtype: float64

## MONTHLY

In [8]:
mkt_monthly = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[0]/100
mkt_monthly = mkt_monthly['Mkt-RF'] + mkt_monthly.RF
mkt_annual.index.name = "month"
mkt_monthly.name = "market monthly"
mkt_monthly.index = mkt_monthly.index.astype(str)
mkt_monthly.tail()

Date
2022-12   -0.0608
2023-01    0.0700
2023-02   -0.0224
2023-03    0.0287
2023-04    0.0096
Name: market monthly, dtype: float64

## DAILY

In [9]:
mkt_daily = pdr('F-F_Research_Data_Factors_Daily','famafrench',start=1926)[0]/100
mkt_daily = mkt_daily['Mkt-RF'] + mkt_daily.RF
mkt_daily.index.name = "day"
mkt_daily.name = "market daily"
mkt_daily.index = mkt_daily.index.astype(str)
mkt_daily.tail()

day
2023-04-24    0.00018
2023-04-25   -0.01742
2023-04-26   -0.00392
2023-04-27    0.01868
2023-04-28    0.00788
Name: market daily, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [10]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


## ANNUAL

In [11]:
price_annual = price.resample("Y").last()
ret_annual = price_annual.pct_change()[:-1]
ret_annual.index = ret_annual.index.map(lambda x: x.year)
ret_annual.index.name = "year"
ret_annual.name = ticker + " annual"
ret_annual.tail()

year
2018   -0.053902
2019    0.889578
2020    0.823067
2021    0.346482
2022   -0.264042
Name: aapl annual, dtype: float64

## MONTHLY

In [15]:
price_monthly = price.resample("M").last()
ret_monthly = price_monthly.pct_change()[:-1]
ret_monthly.index = ret_monthly.index.to_period("M").astype(str)
ret_monthly.index.name = "month"
ret_monthly.name = ticker + " monthly"
ret_monthly.tail()

C:\Users\kerry\AppData\Local\Temp\ipykernel_28000\3697967823.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  ret_monthly.index = ret_monthly.index.to_period("M").astype(str)


month
2023-01    0.110521
2023-02    0.023183
2023-03    0.118649
2023-04    0.028987
2023-05    0.046058
Name: aapl monthly, dtype: float64

## DAILY

In [17]:
ret_daily = price.pct_change()
ret_daily.index = ret_daily.index.to_period("D").astype(str)
ret_daily.index.name = "day"
ret_daily.name = ticker + " daily"
ret_daily.tail()

C:\Users\kerry\AppData\Local\Temp\ipykernel_28000\1185978493.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  ret_daily.index = ret_daily.index.to_period("D").astype(str)


day
2023-05-25    0.006692
2023-05-26    0.014105
2023-05-30    0.010660
2023-05-31   -0.000282
2023-06-01    0.016023
Name: aapl daily, dtype: float64

# SELECT YAHOO RETURN OR MARKET

In [20]:
flag = input("input 0 for market or 1 for yahoo return")
flag = "mkt" if flag=="0" else "ret" 
df_annual = eval(flag + "_annual")
df_monthly = eval(flag + "_monthly")
df_daily = eval(flag + "_daily")

# TABLE

In [21]:
annual = df_annual.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
monthly = df_monthly.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
daily = df_daily.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))
table = pd.concat((daily, annual, monthly), axis=1)
table

,market daily,market annual,market monthly
count,25480.000000,96.000000,1162.000000
mean,0.000420,0.118357,0.009382
std,0.010825,0.200615,0.053373
min,-0.174130,-0.440400,-0.291000
10%,-0.010300,-0.115450,-0.051560
25%,-0.003893,-0.000275,-0.017800
50%,0.000750,0.149250,0.012850
75%,0.005100,0.269525,0.039525
90%,0.010600,0.340750,0.062270
max,0.157620,0.573500,0.389500


# FIGURE 1

In [ ]:
trace = go.Scatter(
    x=data.index.to_list(),
    y=data,
    hovertemplate="%{x}<br>return = %{y:.1%}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title=name,
    template="plotly_white"
)
fig.show()

# FIGURE 2

In [ ]:
trace = go.Box(
    x=data,
    hovertemplate="%{x}<extra></extra>", 
    name=""
)
fig = go.Figure(trace)
fig.update_layout(
    yaxis_title="",
    xaxis_title=name,
    xaxis_tickformat=".0%",
    template="plotly_white",
)
fig.show()

# FIGURE 3

In [ ]:
trace = go.Histogram(
    x=data,
    hovertemplate="%{x}<extra></extra>", 
    name=""
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title=name,
    xaxis_tickformat=".0%",
    template="plotly_white",
)
fig.show()

# FIGURE 4

In [ ]:
trace = go.Scatter(
    x = data.index.to_list(),
    y=(1+data).cumprod(),
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>",
    mode="lines",
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title=name.split()[0] + " Cumulative Return",
    yaxis_tickformat=":.2f",
    template="plotly_white"
)
fig.show()


# FIGURE 5

In [ ]:
trace = go.Scatter(
    x = data.index.to_list(),
    y=(1+data).cumprod(),
    hovertemplate="%{x}<br>accumulation = $%{y:.2f}<extra></extra>",
)
fig = go.Figure(trace)
fig.update_layout(
    yaxis_type="log",
    xaxis_title="Year",
    yaxis_title=name.split()[0] + " Cumulative Return",
    template="plotly_white"
)
fig.show()